This POC focuses on creating a density map, chloropleth of population density based on counties in the US. 

Because we are using jupyter, extensions, must be installed using conda. 

There are two different types of counties df's, 
1. one is from bokeh
2. the other is from a geo json file.

The geo json file is needed to vizualize the data set. 

need to install jupyter extensions in order to show plotly 
https://github.com/plotly/plotly.py/issues/2508


If you see this error, you are not connected to Internet
"<urlopen error [Errno 8] nodename nor servname provided, or not known>"


TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type
https://github.com/numpy/numpy/issues/18165

In [ ]:
from os.path import abspath
import webbrowser
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from bokeh.sampledata.us_counties import data as counties

df = pd.read_csv('../data/external/census_data_popl_2010.csv', encoding="ISO-8859-1")

df = pd.DataFrame(df,
                  columns=
                  ['Target Geo Id2',
                  'Geographic area.1',
                  'Density per square mile of land area - Population'])
                            
df.rename(columns =
          {'Target Geo Id2':'fips',
           'Geographic area.1': 'County',
           'Density per square mile of land area - Population':'Density'},
          inplace = True)

print(f"\nInitial popl data:\n {df.head()}")
print(f"Shape of df = {df.shape}\n")

# Remove non-county rows from data frame.
df = df[df['fips'] > 100]
print(f"Popl data with non-county rows removed:\n {df.head()}")
print(f"Shape of df = {df.shape}\n")

# Create columns for state and county id numbers.
df['state_id'] = (df['fips'] // 1000).astype('int64')
df['cid'] = (df['fips'] % 1000).astype('int64')
print(f"Popl data with new ID columns:\n {df.head()}")
print(f"Shape of df = {df.shape}\n")
print("df info:")
print(df.info())  # Lists data types of each column.

# Check that 5-digit fips handled correctly.
print("\nPopl data at row 500:")
print(df.loc[500])

# Make dictionary of state_id, cid tuple vs popl density.
state_ids = df.state_id.tolist()
cids = df.cid.tolist()
den = df.Density.tolist()
tuple_list = tuple(zip(state_ids, cids))
popl_dens_dict = dict(zip(tuple_list, den))

# Exclude states & territories not part of conterminious US.
EXCLUDED = ('ak', 'hi', 'pr', 'gu', 'vi', 'mp', 'as')

counties = [dict(county, Density=popl_dens_dict[cid])
            for cid, county in counties.items()
            if county["state"] not in EXCLUDED]


In [ ]:
df.head()

In [ ]:
fips_col = df['Density']
fips_col.max()

In [ ]:
df['fips'] = (df['fips']).astype('int64')
df['fips'] = df['fips'].apply(lambda x: '{0:0>5}'.format(x))
df['fips'] = (df['fips']).astype('str')
df_viz = df.drop(columns=['County', 'state_id', 'cid'])
df_viz.head()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
counties["features"][0]

In [ ]:
import plotly.express as px
# import plotly.graph_objs as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected = True)

fig = px.choropleth(df_viz, geojson=counties, locations='fips', color='Density',
                           color_continuous_scale="Viridis",
                           range_color=(0, 500),
                           scope="usa",
                           labels={'Density':'population density'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()
iplot(fig)

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()